## 임베딩

In [ ]:
import os
import uuid
import json
import torch
from pathlib import Path
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels

# ===== 모델 매핑 =====
MODEL_NAME = "bge-ko"   # ← 여기만 바꿔주면 됨 ("e5", "koe5", "bge-ko", "bge-m3", "kure")

MODEL_MAP = {
    "e5": "intfloat/multilingual-e5-small",
    "e5-base": "intfloat/multilingual-e5-base",
    "e5-large": "intfloat/multilingual-e5-large",
    "koe5": "nlpai-lab/KoE5",
    "bge-ko": "dragonkue/bge-m3-ko",
    "bge-m3": "BAAI/bge-m3",
    "kure": "nlpai-lab/KURE-v1",
}

EMBED_MODEL_NAME = MODEL_MAP[MODEL_NAME]
COLLECTION       = "audit_chunks"
BATCH            = 64
QDRANT_PATH      = f"data/vector_store/final-sjchunk/{MODEL_NAME}-qdrant_db"
CHUNK_FILE       = Path("data/structured/enhanced_vector_chunks_9_24.jsonl")

# 토크나이저 포크 경고 끄기 (권장)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# ===== 디바이스 선택 (MPS > CUDA > CPU) =====
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"[INFO] Using device: {device}")

# ===== 모델 로드 =====
model = SentenceTransformer(EMBED_MODEL_NAME, device=device)
dim = model.get_sentence_embedding_dimension()
print(f"[INFO] Loaded model: {MODEL_NAME} ({EMBED_MODEL_NAME}), dim={dim}")

# ===== corpus 로드 =====
assert CHUNK_FILE.exists(), f"청킹 파일을 찾을 수 없습니다: {CHUNK_FILE}"

corpus = []
with open(CHUNK_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        corpus.append(json.loads(line))

print(f"[INFO] Loaded corpus: {len(corpus)} chunks")

client = None
try:
    # ===== Qdrant 연결 (임베디드 모드) =====
    client = QdrantClient(path=QDRANT_PATH)

    # 컬렉션 생성 (존재하지 않을 때만)
    if not client.collection_exists(COLLECTION):
        client.create_collection(
            collection_name=COLLECTION,
            vectors_config=qmodels.VectorParams(
                size=dim,
                distance=qmodels.Distance.COSINE
            ),
            optimizers_config=qmodels.OptimizersConfigDiff(indexing_threshold=20000),
            hnsw_config=qmodels.HnswConfigDiff(m=32, ef_construct=256),
        )
        print(f"[INFO] Created collection: {COLLECTION}")
    else:
        print(f"[INFO] Using existing collection: {COLLECTION}")

    # ===== 업서트 =====
    pending_points = []
    for i in tqdm(range(0, len(corpus), BATCH), desc=f"Upserting ({MODEL_NAME})"):
        batch = corpus[i:i+BATCH]
        texts = [x["text"] for x in batch]

        vecs = model.encode(
            texts,
            normalize_embeddings=True,
            convert_to_numpy=True,
            show_progress_bar=False,
            batch_size=BATCH,
        ).astype("float32")

        pending_points.clear()
        for x, v in zip(batch, vecs):
            pid = str(uuid.uuid4())   # ✅ 무조건 올바른 UUID 생성
            payload = {**x.get("metadata", {}), "text": x["text"]}
            pending_points.append(
                qmodels.PointStruct(id=pid, vector=v.tolist(), payload=payload)
            )


        client.upsert(collection_name=COLLECTION, points=pending_points, wait=False)

    try:
        client.update_collection(
            collection_name=COLLECTION,
            hnsw_config=qmodels.HnswConfigDiff(ef_construct=256),
            optimizers_config=qmodels.OptimizersConfigDiff(default_segment_number=4),
        )
    except Exception:
        pass

    print(f"[INFO] Upsert done: {len(corpus)} → DB path={QDRANT_PATH}")

finally:
    if client is not None:
        try:
            client.close()
        except Exception:
            pass

## Retriever 성능 테스트

In [ ]:
import numpy as np
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
from qdrant_client.http import models as qmodels


# ===== 임베딩 모델 로드 =====
embed_model = SentenceTransformer("dragonkue/bge-m3-ko")

# ===== 숫자만 추출 함수 =====
def extract_numbers(text: str) -> str:
    if text is None:
        return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))

# ===== Retriever 함수 =====
def extract_year_from_query(query: str):
    """질문에서 연도(4자리 숫자) 추출"""
    match = re.search(r"(20\d{2}|19\d{2})", query)
    if match:
        return int(match.group(1))
    return None
    
def dense_search(query: str, model, client, collection_name: str,
                 top_k: int = 3, ground_truth=None):
    """
    Qdrant query_points 기반 Dense Retriever 함수 + 성능지표 계산
    """
    # 0. 질문에서 연도 추출
    year = extract_year_from_query(query)
    
    # 1. 쿼리 임베딩 생성
    qv = model.encode(query, normalize_embeddings=True).tolist()

    # 2. Qdrant 검색 실행 (연도 필터 있으면 적용)
    query_filter = None
    if year:
        query_filter = qmodels.Filter(
            must=[
                qmodels.FieldCondition(
                    key="report_year",
                    match=qmodels.MatchValue(value=year)
                )
            ]
        )

    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter  # ✅ 필터 적용
    )

    # 3. 결과 정리 (payload 전체 반영)
    output = []
    for r in results.points:
        payload = r.payload or {}
        result_item = {
            "score": r.score,
            "text": payload.get("text")
        }
        # metadata 안의 모든 키-값을 추가
        if "metadata" in payload:
            result_item.update(payload["metadata"])
        else:
            # 혹시 metadata 키 없이 flat하게 들어온 경우
            result_item.update(payload)
        output.append(result_item)

    # 4. 성능 지표 계산
    metrics = {}
    if ground_truth:
        # 정답 숫자만 추출
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]

        # 각 검색 결과가 정답과 매칭되는지 여부
        used = set()
        relevances = []
        for r in output:
            nums = extract_numbers(r["text"])
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        # Precision@3
        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        metrics["Precision@3"] = precision

        # Recall@3 (전체 정답 대비 비율)
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        metrics["Recall@3"] = recall

        # F1@3
        if precision + recall > 0:
            metrics["F1@3"] = 2 * (precision * recall) / (precision + recall)
        else:
            metrics["F1@3"] = 0.0

        # MRR
        rr = 0.0
        for rank, rel in enumerate(relevances, 1):
            if rel == 1:
                rr = 1.0 / rank
                break
        metrics["MRR"] = rr

        # nDCG@k
        dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)   # 최대 정답 수
        idcg = sum(1.0 / np.log2(idx + 2) for idx in range(ideal_hits))
        metrics["nDCG@3"] = dcg / idcg if idcg > 0 else 0.0

    return output, metrics


# =========================
# 실행 예시
# =========================
if __name__ == "__main__":
    QDRANT_PATH = "data/vector_store/final-sjchunk/bge-ko-qdrant_db"
    client = QdrantClient(path=QDRANT_PATH)

    collection_name = "audit_chunks"

    questions = [
    "2014년 재무상태표 상 당기 유동자산은 얼마인가?",
    "2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?",
    "2015년 당기 비유동자산은 재무상태표에서 얼마인가?",
    "2015년 손익계산서 상 당기순이익은 얼마인가?",
    "2016년 재무상태표 상 당기 단기금융상품은 얼마인가요?",
    "2016년 포괄손익계산서 상 당기 총포괄이익은 얼마니?",
    "2016년 자본변동표 상 자기주식의 취득은 얼마인가?",
    "2017년 당기 매출채권은 재무상태표에 따르면 얼마냐?",
    "2017년 재무상태표상 전기 현금및현금성자산은 얼마입니까?",
    "2018년 당기 미수금은 재무상태표에서 얼마인가?",
    "2018년 손익계산서상 매출총이익은 얼마인가요?",
    "2019년 재무상태표상 종속기업, 관계기업 및 공동기업 투자는 얼마인가요?",
    "2019년 현금흐름표 상 이익잉여금 배당은 얼마인가요?",
    "2019년 손익계산서상 기본주당이익은 얼마인가요?",
    "2020년 재무상태표 상 자산총계는?",
    "2020년 손익계산서 상 판매비와관리비는 얼마인가요?",
    "2021년 재무상태표상 당기 기타포괄손익-공정가치금융자산은 얼마인가요?",
    "2021년 재무상태표에서 당기 유동비율을 계산하면 얼마인가요?",
    "2021년 손익계산서 상 당기 금융비용은 얼마인가요?",
    "2022년 재무상태표상 당기 비유동부채는 얼마인가?",
    "2022년 손익계산서 상 당기 법인세비용은 얼마니?",
    "2022년 당기 현금흐름표 상 투자활동 현금흐름은 얼마인가?",
    "2023년 재무상태표 상 재고자산은 얼마인가?",
    "2023년 손익계산서 상 당기 영업이익은 얼마인가?",
    "2024년에는 재무상태표상 당기 무형자산이 얼마야?",
    "2024년 재무상태표 상 당기 우선주자본금은 얼마인가?",
    "2024년 손익계산서상 당기 법인세비용은 얼마야?",
    ]
    
    answers = [
        "62,054,773",
        "18,653,817",
        "101,967,575",
        "12,238,469",
        "30,170,656",
        "11,887,806",
        "(7,707,938)",
        "27,881,777",
        "3,778,371",
        "1,515,079",
        "68,715,364",
        "56,571,252",
        "(9,618,210)",
        "2,260",
        "229,664,427",
        "29,038,798",
        "1,662,532",
        "1.38",
        "3,698,675",
        "4,581,512",
        "4,273,142",
        "(28,123,886)",
        "29,338,151",
        "(11,526,297)",
        "10,496,956",
        "119,467",
        "(1,832,987)",
    ]

    all_metrics = []

    for q, a in zip(questions, answers):
        results, metrics = dense_search(
            query=q,
            model=embed_model,
            client=client,
            collection_name=collection_name,
            top_k=3,
            ground_truth=[a]
        )

        print("=" * 100)
        print(f"질문: {q}")

        # Top-3 결과 모두 출력
        if results:
            print("\n[검색 결과 Top-3]")
            for rank, r in enumerate(results, 1):
                print(f"{rank}위: {r['text']}")
                print(f"   출처: {r['report_year']}년, (score={r['score']:.4f})\n")
        else:
            print("검색 결과 없음")

        print("\n[정답]")
        print(a)

        print("\n[성능 지표]")
        print(
            f" Precision@3={metrics.get('Precision@3', 0):.2f},"
            f" Recall@3={metrics.get('Recall@3', 0):.2f},"
            f" F1@3={metrics.get('F1@3', 0):.2f},"
            f" nDCG@3={metrics.get('nDCG@3', 0):.2f},"
            f" MRR={metrics.get('MRR', 0):.2f}"
        )

        all_metrics.append(metrics)

    # 평균 성능 지표
    df = pd.DataFrame(all_metrics)
    print("\n" + "=" * 100)
    print("=== 평균 성능 지표 ===")
    print(df.mean().round(3))

    client.close()


질문: 2014년 재무상태표 상 당기 유동자산은 얼마인가?

[검색 결과 Top-3]
1위: 재무상태표에서 2014년 (당기) 유동자산는 62,054,773백만원입니다.
   출처: 2014년, (score=0.8690)

2위: 재무상태표에서 2014년 (당기) 비유동자산는 102,005,810백만원입니다.
   출처: 2014년, (score=0.7909)

3위: 재무상태표에서 2014년 (당기) 기타유동자산는 821,079백만원입니다.
   출처: 2014년, (score=0.7762)


[정답]
62,054,773

[성능 지표]
 Precision@3=0.33, Recall@3=1.00, F1@3=0.50, nDCG@3=1.00, MRR=1.00
질문: 2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?

[검색 결과 Top-3]
1위: 현금흐름표에서 2014년 (당기) 영업활동현금흐름는 18,653,817백만원입니다.
   출처: 2014년, (score=0.8249)

2위: 현금흐름표에서 2014년 (당기) 투자활동현금흐름는 -15,951,220백만원입니다.
   출처: 2014년, (score=0.7903)

3위: 현금흐름표에서 2014년 (당기) 재무활동현금흐름는 -3,089,586백만원입니다.
   출처: 2014년, (score=0.7843)


[정답]
18,653,817

[성능 지표]
 Precision@3=0.33, Recall@3=1.00, F1@3=0.50, nDCG@3=1.00, MRR=1.00
질문: 2015년 당기 비유동자산은 재무상태표에서 얼마인가?

[검색 결과 Top-3]
1위: 재무상태표에서 2015년 (당기) 비유동자산는 101,967,575백만원입니다.
   출처: 2015년, (score=0.8659)

2위: 재무상태표에서 2015년 (당기) 기타비유동자산는 1,331,840백만원입니다.
   출처: 2015년, (score=0.7951)

3위: 재무상태표에서 2015년 (당기) 비유동부채는 

## RAG

In [1]:
import os
import re
import uuid
import json
import torch
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from llama_cpp import Llama
import numpy as np
from langchain_community.llms import LlamaCpp
from ragas.llms import LangchainLLMWrapper

# RAGAS
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import evaluate

# ===== 임베딩 모델 로드 =====
MODEL_NAME = "bge-m3-ko"
EMBED_MODEL_NAME = "dragonkue/bge-m3-ko"
COLLECTION = "audit_chunks"

# ===== Qdrant 클라이언트 =====
QDRANT_PATH = "data/vector_store/final-sjchunk/bge-ko-qdrant_db"
client = QdrantClient(path=QDRANT_PATH)

# 토크나이저 포크 경고 끄기 (권장)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# ===== 디바이스 선택 (MPS > CUDA > CPU) =====
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"[INFO] Using device: {device}")


embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=device)
print("✅ 임베딩 모델 로드 완료!")

# ===== dense_search 함수 import =====
# (예빈씨가 이전에 정의한 dense_search 그대로 사용한다고 가정)
def extract_year_from_query(query: str):
    """질문에서 연도(4자리 숫자) 추출"""
    match = re.search(r"(20\d{2}|19\d{2})", query)
    if match:
        return int(match.group(1))
    return None

def extract_numbers(text: str) -> str:
    """텍스트에서 숫자만 추출"""
    if text is None:
        return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))
    
def dense_search(query: str, model, client, collection_name: str,
                 top_k: int = 3, ground_truth=None, debug: bool = False):
    """
    Qdrant query_points 기반 Dense Retriever 함수 + 성능지표 계산 (+ 디버그 로그 옵션)
    """
    year = extract_year_from_query(query)
    qv = model.encode(query, normalize_embeddings=True).tolist()

    if isinstance(ground_truth, str):
        ground_truth = [ground_truth]
    
    query_filter = None
    if year:
        query_filter = qmodels.Filter(
            must=[qmodels.FieldCondition(
                key="report_year",
                match=qmodels.MatchValue(value=year)
            )]
        )

    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=top_k,
        with_payload=True,
        query_filter=query_filter
    )

    output = []
    for r in results.points:
        payload = r.payload or {}
        result_item = {"score": r.score, "text": payload.get("text")}
        if "metadata" in payload:
            result_item.update(payload["metadata"])
        else:
            result_item.update(payload)
        output.append(result_item)

    metrics = {}
    if ground_truth:
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]
        used = set()
        relevances = []

        for r in output:
            nums = extract_numbers(r["text"])
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        metrics["Precision@3"] = precision
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        metrics["Recall@3"] = recall
        metrics["F1@3"] = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0
        rr = 0.0
        for rank, rel in enumerate(relevances, 1):
            if rel == 1:
                rr = 1.0 / rank
                break
        metrics["MRR"] = rr
        dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)
        idcg = sum(1.0 / np.log2(idx + 2) for idx in range(ideal_hits))
        metrics["nDCG@3"] = dcg / idcg if idcg > 0 else 0.0

    return output, metrics
    
# ===== LLM 모델 로드 =====
model_path = Path("/Users/bag-yebin/Desktop/흠/자연어처리/samsun-audit-rag-qa/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf").resolve()

llm = Llama(
    model_path=str(model_path),
    n_ctx=8192,
    n_threads=8,
    n_gpu_layers=35
)
print("✅ LLM 모델 로드 완료!")

# ===== RAG Pipeline =====
def rag_pipeline(query: str, model, client, collection_name: str = COLLECTION, top_k: int = 3, ground_truth=None):
    # 1) Retriever 단계
    results, metrics = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k,
        ground_truth=ground_truth  

    )

    # 2) 검색 결과 합치기
    context_texts = [r["text"] for r in results if r.get("text")]
    context_text = "\n".join(context_texts)

    # 3) Reader 호출 (Zephyr LLM) - 튜닝된 프롬프트 적용
    prompt = f"""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    너는 재무보고서 전문가이자 **숫자 추출기**다. 
    검색된 문서와 metadata만 근거로 반드시 답해야 한다. 
    규칙:
    1. 정답은 반드시 숫자(예: 62,054,773) 또는 "문서에 없음" 만 출력한다. 
    2. 단위(백만원, 억원 등)는 출력하지 않는다. 
    3. 계산이 필요한 경우 (예: 비율) 정확히 계산하고 소수점 둘째 자리까지 출력한다.
    4. 검색된 문서에 정답이 없으면 반드시 "문서에 없음"이라고 출력한다.
    5. 불필요한 설명, 해석, 텍스트는 절대 쓰지 말고, **최종 숫자만 출력**한다.
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    검색된 문서:
    {context_text}
    
    질문: {query}
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """

    response = llm(prompt, max_tokens=512, stop=["<|eot_id|>"])
    answer = response["choices"][0]["text"].strip()

    # ✅ "없음" 케이스 예외 처리
    if ground_truth and ground_truth[0] == "-":
        if "없음" in answer:
            metrics = {"Precision@3": 1.0, "Recall@3": 1.0, "F1@3": 1.0, "MRR": 1.0, "nDCG@3": 1.0}
        else:
            metrics = {"Precision@3": 0.0, "Recall@3": 0.0, "F1@3": 0.0, "MRR": 0.0, "nDCG@3": 0.0}


    return answer, metrics, context_texts   # ✅ contexts도 반환
# ===== 실행 + RAGAS 평가 =====
if __name__ == "__main__":
    # 질문 & 정답 데이터셋
    questions = [
    "2014년 재무상태표 상 당기 유동자산은 얼마인가?",
    "2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?",
    "2015년 당기 비유동자산은 재무상태표에서 얼마인가?",
    "2015년 손익계산서 상 당기순이익은 얼마인가?",
    "2016년 재무상태표 상 당기 단기금융상품은 얼마인가요?",
    "2016년 포괄손익계산서 상 당기 총포괄이익은 얼마니?",
    "2016년 자본변동표 상 자기주식의 취득은 얼마인가?",
    "2017년 당기 매출채권은 재무상태표에 따르면 얼마냐?",
    "2017년 재무상태표상 전기 현금및현금성자산은 얼마입니까?",
    "2017년 재무상태표상 당기 매각예정분류자산은 얼마인가요?",
    "2018년 당기 미수금은 재무상태표에서 얼마인가?",
    "2018년 손익계산서상 매출총이익은 얼마인가요?",
    "2019년 재무상태표상 종속기업, 관계기업 및 공동기업 투자는 얼마인가요?",
    "2019년 현금흐름표 상 이익잉여금 배당은 얼마인가요?",
    "2019년 손익계산서상 기본주당이익은 얼마인가요?",
    "2020년 재무상태표 상 자산총계는?",
    "2020년 손익계산서 상 판매비와관리비는 얼마인가요?",
    "2021년 재무상태표상 당기 기타포괄손익-공정가치금융자산은 얼마인가요?",
    "2021년 재무상태표에서 당기 유동비율을 계산하면 얼마인가요?",
    "2021년 손익계산서 상 당기 금융비용은 얼마인가요?",
    "2022년 재무상태표상 당기 비유동부채는 얼마인가?",
    "2022년 손익계산서 상 당기 법인세비용은 얼마니?",
    "2022년 당기 현금흐름표 상 투자활동 현금흐름은 얼마인가?",
    "2023년 재무상태표 상 재고자산은 얼마인가?",
    "2023년 당기 영업이익은 얼마인가?",
    "2024년에는 재무상태표상 당기 무형자산이 얼마야?",
    "2024년 재무상태표 상 당기 우선주자본금은 얼마인가?",
    "2024년 손익계산서상 당기 법인세비용은 얼마야?",
    ]
    
    answers = [
        "62,054,773",
        "18,653,817",
        "101,967,575",
        "12,238,469",
        "30,170,656",
        "11,887,806",
        "(7,707,938)",
        "27,881,777",
        "3,778,371",
        "-",
        "1,515,079",
        "68,715,364",
        "56,571,252",
        "(9,618,210)",
        "2,260",
        "229,664,427",
        "29,038,798",
        "1,662,532",
        "1.38",
        "3,698,675",
        "4,581,512",
        "4,273,142",
        "(28,123,886)",
        "29,338,151",
        "(11,526,297)",
        "10,496,956",
        "119,467",
        "(1,832,987)",
    ]
    
    qa_pairs = [{"question": q, "ground_truth": [a]} for q, a in zip(questions, answers)]

    eval_data = {"question": [], "answer": [], "contexts": [], "reference": []}
    metrics_list = []   # ✅ 성능지표 저장 리스트 추가

    for qa in tqdm(qa_pairs, desc="RAG 평가 진행중", unit="질문"):
        q = qa["question"]
        gt = qa["ground_truth"][0]

        answer, metrics, contexts = rag_pipeline(q, embed_model, client, COLLECTION, ground_truth=gt)
        eval_data["question"].append(q)
        eval_data["answer"].append(answer)
        eval_data["contexts"].append(contexts)
        eval_data["reference"].append(gt)   # RAGAS가 요구하는 필드명

        metrics_list.append(metrics)  # ✅ 각 성능지표 저장

        print("=" * 80)
        print("질문:", q)
        print("생성 답변:", answer)
        print("정답:", gt)
        print("Retriever 성능지표:", metrics)

    dataset = Dataset.from_dict(eval_data)

    # ✅ 성능지표 평균 계산
    df_metrics = pd.DataFrame(metrics_list)
    print("\n=== 평균 성능 지표 ===")
    print(df_metrics.mean())


/opt/anaconda3/envs/parsing_conda/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[INFO] Using device: mps


You try to use a model that was created with version 3.1.0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/opt/anaconda3/envs/parsing_conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
llama_model_load_from_file_impl: using device Metal (Apple M3) - 7886 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /Users/bag-yebin/Desktop/흠/자연어처리/samsun-audit-rag-qa/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loade

✅ 임베딩 모델 로드 완료!


load: control token: 128162 '<|reserved_special_token_154|>' is not marked as EOG
load: control token: 128165 '<|reserved_special_token_157|>' is not marked as EOG
load: control token: 128057 '<|reserved_special_token_49|>' is not marked as EOG
load: control token: 128050 '<|reserved_special_token_42|>' is not marked as EOG
load: control token: 128056 '<|reserved_special_token_48|>' is not marked as EOG
load: control token: 128230 '<|reserved_special_token_222|>' is not marked as EOG
load: control token: 128098 '<|reserved_special_token_90|>' is not marked as EOG
load: control token: 128153 '<|reserved_special_token_145|>' is not marked as EOG
load: control token: 128084 '<|reserved_special_token_76|>' is not marked as EOG
load: control token: 128082 '<|reserved_special_token_74|>' is not marked as EOG
load: control token: 128102 '<|reserved_special_token_94|>' is not marked as EOG
load: control token: 128253 '<|reserved_special_token_245|>' is not marked as EOG
load: control token: 12

...

질문: 2024년에는 재무상태표상 당기 무형자산이 얼마야?
생성 답변: 10,496,956
정답: 10,496,956
Retriever 성능지표: {'Precision@3': 0.3333333333333333, 'Recall@3': 1.0, 'F1@3': 0.5, 'MRR': 1.0, 'nDCG@3': 1.0}


llama_perf_context_print:        load time =    6427.52 ms
llama_perf_context_print: prompt eval time =     893.89 ms /   105 tokens (    8.51 ms per token,   117.46 tokens per second)
llama_perf_context_print:        eval time =     254.61 ms /     4 runs   (   63.65 ms per token,    15.71 tokens per second)
llama_perf_context_print:       total time =    1154.71 ms /   109 tokens
llama_perf_context_print:    graphs reused =          3
RAG 평가 진행중:  96%|█████████▋| 27/28 [00:50<00:01,  1.89s/질문]Llama.generate: 212 prefix-match hit, remaining 127 prompt tokens to eval


질문: 2024년 재무상태표 상 당기 우선주자본금은 얼마인가?
생성 답변: 119,467
정답: 119,467
Retriever 성능지표: {'Precision@3': 0.3333333333333333, 'Recall@3': 1.0, 'F1@3': 0.5, 'MRR': 1.0, 'nDCG@3': 1.0}


llama_perf_context_print:        load time =    6427.52 ms
llama_perf_context_print: prompt eval time =     895.68 ms /   127 tokens (    7.05 ms per token,   141.79 tokens per second)
llama_perf_context_print:        eval time =     381.14 ms /     6 runs   (   63.52 ms per token,    15.74 tokens per second)
llama_perf_context_print:       total time =    1286.10 ms /   133 tokens
llama_perf_context_print:    graphs reused =          5
RAG 평가 진행중: 100%|██████████| 28/28 [00:52<00:00,  1.86s/질문]

질문: 2024년 손익계산서상 당기 법인세비용은 얼마야?
생성 답변: -1,832,987
정답: (1,832,987)
Retriever 성능지표: {'Precision@3': 0.3333333333333333, 'Recall@3': 1.0, 'F1@3': 0.5, 'MRR': 1.0, 'nDCG@3': 1.0}

=== 평균 성능 지표 ===
Precision@3    0.333333
Recall@3       0.928571
F1@3           0.482143
MRR            0.910714
nDCG@3         0.915390
dtype: float64


In [ ]:
#CrossEndcoder 리랭커 실험

import os
import re
import uuid
import json
import torch
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder  # ✅ 추가: 리랭커
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from llama_cpp import Llama
import numpy as np
from langchain_community.llms import LlamaCpp
from ragas.llms import LangchainLLMWrapper

# RAGAS
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import evaluate

# ===== TOP-K 전역 설정 (여기만 바꾸면 전체에 반영) =====
TOP_K = 3  # ← 1/3/5 등 원하는 k로 변경

# ===== 임베딩 모델 로드 =====
MODEL_NAME = "bge-m3-ko"
EMBED_MODEL_NAME = "dragonkue/bge-m3-ko"
COLLECTION = "audit_chunks"

# ===== Qdrant 클라이언트 =====
QDRANT_PATH = "data/vector_store/final-sjchunk/bge-ko-qdrant_db"
client = QdrantClient(path=QDRANT_PATH)

# 토크나이저 포크 경고 끄기 (권장)
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# ===== 디바이스 선택 (MPS > CUDA > CPU) =====
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"[INFO] Using device: {device}")

embed_model = SentenceTransformer(EMBED_MODEL_NAME, device=device)
print("✅ 임베딩 모델 로드 완료!")
# ===== (옵션) 리랭커 로드 설정 =====
USE_RERANKER = True                 # ✅ 기본은 끔 (기존과 동일 동작)
RERANKER_MODEL = "BAAI/bge-reranker-base"
CANDIDATES_K = 50                    # ✅ 리랭킹 전에 넓게 뽑아올 개수

reranker = None
if USE_RERANKER:
    if torch.cuda.is_available():
        ce_device = "cuda"
    elif torch.backends.mps.is_available():
        ce_device = "mps"
    else:
        ce_device = "cpu"
    reranker = CrossEncoder(RERANKER_MODEL, device=ce_device)
    print(f"✅ 리랭커 로드 완료! ({RERANKER_MODEL}, device={ce_device})")

# ===== dense_search 함수 =====
def extract_year_from_query(query: str):
    match = re.search(r"(20\d{2}|19\d{2})", query)
    if match:
        return int(match.group(1))
    return None

def extract_numbers(text: str) -> str:
    if text is None:
        return ""
    return "".join(re.findall(r"\d+", text.replace(",", "")))
    
def dense_search(
    query: str,
    model,
    client,
    collection_name: str,
    top_k: int = TOP_K,              # ✅ 기본값: TOP_K
    ground_truth=None,
    debug: bool = False,
    use_reranker: bool = False,
    reranker_model=None,
    candidates_k: int = None
):
    """
    Qdrant query_points 기반 Dense Retriever 함수 + (옵션) 리랭커 + 성능지표 계산
    """
    year = extract_year_from_query(query)
    qv = model.encode(query, normalize_embeddings=True).tolist()

    if isinstance(ground_truth, str):
        ground_truth = [ground_truth]
    
    query_filter = None
    if year:
        query_filter = qmodels.Filter(
            must=[qmodels.FieldCondition(
                key="report_year",
                match=qmodels.MatchValue(value=year)
            )]
        )

    # ✅ 리랭커 켜면 candidates_k만큼 넓게 뽑기, 아니면 top_k만
    limit_n = candidates_k if (use_reranker and (candidates_k is not None)) else top_k

    results = client.query_points(
        collection_name=collection_name,
        query=qv,
        limit=limit_n,
        with_payload=True,
        query_filter=query_filter
    )

    # 1차 결과 수집
    raw = []
    for r in results.points:
        payload = r.payload or {}
        item = {"score": r.score, "text": payload.get("text")}
        if "metadata" in payload:
            item.update(payload["metadata"])
        else:
            item.update(payload)
        raw.append(item)

    # ✅ (옵션) 리랭킹
    if use_reranker and reranker_model is not None and len(raw) > 0:
        pairs = [(query, doc.get("text", "") or "") for doc in raw]
        ce_scores = reranker_model.predict(pairs, convert_to_numpy=True)
        for i, s in enumerate(ce_scores):
            raw[i]["ce_score"] = float(s)
        raw.sort(key=lambda x: x.get("ce_score", 0.0), reverse=True)
        output = raw[:top_k]
    else:
        output = raw[:top_k]

    # ===== 성능지표 (Retriever 기준) =====
    metrics = {}
    if ground_truth:
        normalized_gt = [extract_numbers(gt) for gt in ground_truth]
        used = set()
        relevances = []

        for r in output:
            nums = extract_numbers(r.get("text", ""))
            hit = 0
            for gt in normalized_gt:
                if gt and gt in nums and gt not in used:
                    hit = 1
                    used.add(gt)
                    break
            relevances.append(hit)

        # ✅ 메트릭 키를 TOP_K에 맞춰 동적으로 부여
        at_key = f"@{top_k}"
        precision = sum(relevances) / top_k if top_k > 0 else 0.0
        metrics[f"Precision{at_key}"] = precision
        recall = min(sum(relevances), len(normalized_gt)) / len(normalized_gt) if normalized_gt else 0.0
        metrics[f"Recall{at_key}"] = recall
        metrics[f"F1{at_key}"] = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0

        rr = 0.0
        for rank, rel in enumerate(relevances, 1):
            if rel == 1:
                rr = 1.0 / rank
                break
        metrics["MRR"] = rr

        dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevances))
        ideal_hits = min(len(normalized_gt), top_k)
        idcg = sum(1.0 / np.log2(idx + 2) for idx in range(ideal_hits))
        metrics[f"nDCG{at_key}"] = dcg / idcg if idcg > 0 else 0.0

    return output, metrics

# ===== LLM 모델 로드 =====
model_path = Path("/Users/bag-yebin/Desktop/흠/자연어처리/samsun-audit-rag-qa/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf").resolve()

llm = Llama(
    model_path=str(model_path),
    n_ctx=8192,
    n_threads=8,
    n_gpu_layers=35
)
print("✅ LLM 모델 로드 완료!")

# ===== RAG Pipeline =====
def rag_pipeline(
    query: str,
    model,
    client,
    collection_name: str = COLLECTION,
    top_k: int = TOP_K,              # ✅ 기본값: TOP_K
    ground_truth=None,
    use_reranker: bool = False,
    reranker_model=None,
    candidates_k: int = None
):
    # 1) Retriever 단계 (+옵션 리랭커)
    results, metrics = dense_search(
        query=query,
        model=model,
        client=client,
        collection_name=collection_name,
        top_k=top_k,
        ground_truth=ground_truth,
        use_reranker=use_reranker,
        reranker_model=reranker_model,
        candidates_k=candidates_k
    )

    # 2) 검색 결과 합치기
    context_texts = [r.get("text") for r in results if r.get("text")]
    context_text = "\n".join(context_texts)

    # 3) Reader 호출
    prompt = f"""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    너는 재무보고서 전문가다. 
    검색된 문서와 metadata만 근거로 답해야 한다. 
    없는 값은 반드시 "문서에 없음"이라고 하라. 
    추측하거나 새로 만들어내지 마라. 
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    검색된 문서:
    {context_text}
    
    질문: {query}
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    """

    response = llm(prompt, max_tokens=512, stop=["<|eot_id|>"])
    answer = response["choices"][0]["text"].strip()

    # ✅ "없음" 케이스 예외 처리 (키도 TOP_K에 맞춰 부여)
    if ground_truth and isinstance(ground_truth, list) and len(ground_truth) > 0 and ground_truth[0] == "-":
        at_key = f"@{top_k}"
        if "없음" in answer:
            metrics = {f"Precision{at_key}": 1.0, f"Recall{at_key}": 1.0, f"F1{at_key}": 1.0, "MRR": 1.0, f"nDCG{at_key}": 1.0}
        else:
            metrics = {f"Precision{at_key}": 0.0, f"Recall{at_key}": 0.0, f"F1{at_key}": 0.0, "MRR": 0.0, f"nDCG{at_key}": 0.0}

    return answer, metrics, context_texts

# ===== 실행 + 평가 루프 (그대로) =====
if __name__ == "__main__":
    # 질문 & 정답 데이터셋
    questions = [
    "2014년 재무상태표 상 당기 유동자산은 얼마인가?",
    "2014년 현금흐름표 상 당기 영업활동 현금흐름은 얼마인가?",
    "2015년 당기 비유동자산은 재무상태표에서 얼마인가?",
    "2015년 손익계산서 상 당기순이익은 얼마인가?",
    "2016년 재무상태표 상 당기 단기금융상품은 얼마인가요?",
    "2016년 포괄손익계산서 상 당기 총포괄이익은 얼마니?",
    "2016년 자본변동표 상 자기주식의 취득은 얼마인가?",
    "2017년 당기 매출채권은 재무상태표에 따르면 얼마냐?",
    "2017년 재무상태표상 전기 현금및현금성자산은 얼마입니까?",
    "2017년 재무상태표상 당기 매각예정분류자산은 얼마인가요?",
    "2018년 당기 미수금은 재무상태표에서 얼마인가?",
    "2018년 손익계산서상 매출총이익은 얼마인가요?",
    "2019년 재무상태표상 종속기업, 관계기업 및 공동기업 투자는 얼마인가요?",
    "2019년 현금흐름표 상 이익잉여금 배당은 얼마인가요?",
    "2019년 손익계산서상 기본주당이익은 얼마인가요?",
    "2020년 재무상태표 상 자산총계는?",
    "2020년 손익계산서 상 판매비와관리비는 얼마인가요?",
    "2021년 재무상태표상 당기 기타포괄손익-공정가치금융자산은 얼마인가요?",
    "2021년 재무상태표에서 당기 유동비율을 계산하면 얼마인가요?",
    "2021년 손익계산서 상 당기 금융비용은 얼마인가요?",
    "2022년 재무상태표상 당기 비유동부채는 얼마인가?",
    "2022년 손익계산서 상 당기 법인세비용은 얼마니?",
    "2022년 당기 현금흐름표 상 투자활동 현금흐름은 얼마인가?",
    "2023년 재무상태표 상 재고자산은 얼마인가?",
    "2023년 당기 영업이익은 얼마인가?",
    "2024년에는 재무상태표상 당기 무형자산이 얼마야?",
    "2024년 재무상태표 상 당기 우선주자본금은 얼마인가?",
    "2024년 손익계산서상 당기 법인세비용은 얼마야?",
    ]
    
    answers = [
        "62,054,773",
        "18,653,817",
        "101,967,575",
        "12,238,469",
        "30,170,656",
        "11,887,806",
        "(7,707,938)",
        "27,881,777",
        "3,778,371",
        "-",
        "1,515,079",
        "68,715,364",
        "56,571,252",
        "(9,618,210)",
        "2,260",
        "229,664,427",
        "29,038,798",
        "1,662,532",
        "1.38",
        "3,698,675",
        "4,581,512",
        "4,273,142",
        "(28,123,886)",
        "29,338,151",
        "(11,526,297)",
        "10,496,956",
        "119,467",
        "(1,832,987)",
    ]
    
    qa_pairs = [{"question": q, "ground_truth": [a]} for q, a in zip(questions, answers)]

    eval_data = {"question": [], "answer": [], "contexts": [], "reference": []}
    metrics_list = []   # ✅ 성능지표 저장 리스트

    for qa in tqdm(qa_pairs, desc="RAG 평가 진행중", unit="질문"):
        q = qa["question"]
        gt = qa["ground_truth"][0]

        # ✅ TOP_K 한 번에 적용
        answer, metrics, contexts = rag_pipeline(
            q,
            embed_model,
            client,
            COLLECTION,
            top_k=TOP_K,                 # ← 여기만 TOP_K로
            ground_truth=[gt],
            use_reranker=USE_RERANKER,
            reranker_model=reranker,
            candidates_k=CANDIDATES_K
        )

        eval_data["question"].append(q)
        eval_data["answer"].append(answer)
        eval_data["contexts"].append(contexts)
        eval_data["reference"].append(gt)

        metrics_list.append(metrics)

        print("=" * 80)
        print("질문:", q)
        print("생성 답변:", answer)
        print("정답:", gt)
        print("Retriever 성능지표:", metrics

    dataset = Dataset.from_dict(eval_data)

    # ✅ 성능지표 평균 계산
    df_metrics = pd.DataFrame(metrics_list)
    print("\n=== 평균 성능 지표 ===")
    print(df_metrics.mean())


/opt/anaconda3/envs/parsing_conda/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[INFO] Using device: mps


You try to use a model that was created with version 3.1.0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/opt/anaconda3/envs/parsing_conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ 임베딩 모델 로드 완료!


/opt/anaconda3/envs/parsing_conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
llama_model_load_from_file_impl: using device Metal (Apple M3) - 7886 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /Users/bag-yebin/Desktop/흠/자연어처리/samsun-audit-rag-qa/models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Ins

...

질문: 2024년에는 재무상태표상 당기 무형자산이 얼마야?
생성 답변: 재무상태표상 당기 무형자산은 10,496,956백만원입니다.
정답: 10,496,956
Retriever 성능지표: {'Precision@3': 0.3333333333333333, 'Recall@3': 1.0, 'F1@3': 0.5, 'MRR': 0.5, 'nDCG@3': 0.6309297535714575}


Llama.generate: 97 prefix-match hit, remaining 109 prompt tokens to eval
llama_perf_context_print:        load time =    7474.82 ms
llama_perf_context_print: prompt eval time =    1178.92 ms /   109 tokens (   10.82 ms per token,    92.46 tokens per second)
llama_perf_context_print:        eval time =    2421.93 ms /    29 runs   (   83.51 ms per token,    11.97 tokens per second)
llama_perf_context_print:       total time =    3615.08 ms /   138 tokens
llama_perf_context_print:    graphs reused =         27
RAG 평가 진행중:  96%|█████████▋| 27/28 [01:26<00:03,  3.17s/질문]

질문: 2024년 재무상태표 상 당기 우선주자본금은 얼마인가?
생성 답변: 재무상태표에서 2024년 (당기) 우선주자본금은 119,467백만원입니다.
정답: 119,467
Retriever 성능지표: {'Precision@3': 0.3333333333333333, 'Recall@3': 1.0, 'F1@3': 0.5, 'MRR': 1.0, 'nDCG@3': 1.0}


Llama.generate: 83 prefix-match hit, remaining 126 prompt tokens to eval
llama_perf_context_print:        load time =    7474.82 ms
llama_perf_context_print: prompt eval time =    1104.63 ms /   126 tokens (    8.77 ms per token,   114.07 tokens per second)
llama_perf_context_print:        eval time =     395.04 ms /     6 runs   (   65.84 ms per token,    15.19 tokens per second)
llama_perf_context_print:       total time =    1503.26 ms /   132 tokens
llama_perf_context_print:    graphs reused =          5
RAG 평가 진행중: 100%|██████████| 28/28 [01:29<00:00,  3.19s/질문]

질문: 2024년 손익계산서상 당기 법인세비용은 얼마야?
생성 답변: 1,832,987
정답: (1,832,987)
Retriever 성능지표: {'Precision@3': 0.3333333333333333, 'Recall@3': 1.0, 'F1@3': 0.5, 'MRR': 1.0, 'nDCG@3': 1.0}

=== 평균 성능 지표 ===
Precision@3    0.285714
Recall@3       0.785714
F1@3           0.410714
MRR            0.684524
nDCG@3         0.710457
dtype: float64


## RAGAS

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
import os
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# --- LLM (OpenAI GPT-4o-mini 같은 최신 모델 추천) ---
llm_langchain = ChatOpenAI(
    model="gpt-4o-mini",   # 또는 "gpt-4o", "gpt-4-turbo"
    temperature=0,
    max_tokens=512
)
ragas_llm = LangchainLLMWrapper(llm_langchain)

# --- Embeddings (OpenAI 기본 임베딩) ---
ragas_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model="text-embedding-3-small")  # 또는 "text-embedding-3-large"
)

# --- RAGAS 평가 실행 ---
result = evaluate(
    dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
    llm=ragas_llm,
    embeddings=ragas_embeddings,
    batch_size=2
)

print(result)

/var/folders/8l/bz01tth90yv2m27jxhbcncfw0000gn/T/ipykernel_12194/4160336032.py:18: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(


Evaluating:   0%|          | 0/112 [00:00<?, ?it/s]

Batch 1/56:   0%|          | 0/2 [00:00<?, ?it/s]

{'faithfulness': 0.8690, 'answer_relevancy': 0.1014, 'context_precision': 0.8690, 'context_recall': 0.8571}
